In [11]:
import os
import re
import uuid
import shutil
from pathlib import Path

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [6]:
import wombat.bsub as bsub

###### batch cwl on compute1

In [20]:
run_dir = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl'
log_dir = os.path.join(run_dir, 'logs')

In [19]:
n_jobs = 5
test_dir_root = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl'
inputs_fps, dconfigs, run_names = [], [], list([str(x) for x in range(n_jobs)])
for i in run_names:
    input_yaml = f'../tests/data/batch_cwl/inputs/hello.{i}.yaml'
    shutil.copy('/home/estorrs/pecgs-pipeline/cwl/hello/template.hello.yaml',
                input_yaml)
    workflow_root = os.path.join(run_dir, 'i')
    template_fp = f'../tests/data/batch_cwl/inputs/{i}.cromwell-config-db.compute1.dat'
    bsub.save_compute1_cromwell_template(workflow_root, template_fp)
    
    inputs_fps.append(os.path.join(test_dir_root, 'inputs', f'hello.{i}.yaml'))
    dconfigs.append(os.path.join(test_dir_root, 'inputs', f'{i}.cromwell-config-db.compute1.dat'))

shutil.copy('../wombat/templates/server-cromwell-config.compute1.dat',
            '../tests/data/batch_cwl/inputs/server-cromwell-config.compute1.dat')
server_config = os.path.join(test_dir_root, 'inputs', 'server-cromwell-config.compute1.dat')
    
inputs_fps, dconfigs

(['/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/hello.0.yaml',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/hello.1.yaml',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/hello.2.yaml',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/hello.3.yaml',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/hello.4.yaml'],
 ['/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/0.cromwell-config-db.compute1.dat',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/1.cromwell-config-db.compute1.dat',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/2.cromwell-config-db.compute1.dat',
  '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/3.cromwell-config-db.compute1.dat',
  '/storage1/fs1/dinglab

In [22]:
tool_root = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'
cwl_fp = os.path.join(tool_root, 'cwl', 'hello', 'hello.cwl')
volumes = [test_dir_root, tool_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active', '/scratch1/fs1/dinglab/estorrs']


In [41]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell_hello'
args['n_concurrent'] = 25
args

{'mem': 10,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell_hello',
 'n_concurrent': 25,
 'interactive': False,
 'username': 'estorrs'}

In [46]:
start_commands, cromwell_server_command, run_commands = bsub.batch_cromwell_commands(
    dconfigs, server_config, cwl_fp, inputs_fps, run_names, log_dir, run_dir, args, volumes)
filepath = os.path.join('../tests/data/batch_cwl/1.start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/batch_cwl/2.start_cromwell.sh')
bsub.write_command_file([cromwell_server_command], filepath)
filepath = os.path.join('../tests/data/batch_cwl/3.run_jobs.sh')
bsub.write_command_file(run_commands, filepath)

In [50]:
for c in start_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/logs
mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/0/cromwell-workdir/logs
mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/1/cromwell-workdir/logs
mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/2/cromwell-workdir/logs
mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/3/cromwell-workdir/logs
mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/4/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /storage1/fs1/dinglab/Active:/storage1/fs1

In [48]:
cromwell_server_command

'/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/server-cromwell-config.compute1.dat -jar /usr/local/cromwell/cromwell-47.jar server'

In [49]:
run_commands

["bsub -q dinglab -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/test_cromwell_hello -J cromwell_launch_0 -oo /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/logs/0.log '/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/0.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/batch_cwl/inputs/hello.0.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/cwl/hello/hello.cwl'",
 "bsub -q dinglab -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/test_cromwell_hello -J cromwell_launch_1 -oo /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/batch_cwl/logs/1.log '/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projec